# Doctor's Assistant for preliminary melanoma detection

In [124]:
import tensorflow as tf
print(tf.__version__)
# !pip uninstall tensorflow

2.17.0


In [125]:
import keras
print(keras.__version__)

3.4.1


In [126]:
!pip install gradio transformers torch
!pip install google-cloud-storage
!pip install langdetect

In [127]:
import gradio as gr
from transformers import pipeline
import spacy
import numpy as np
import tensorflow as tf
from PIL import Image
import requests
from io import BytesIO
from langdetect import detect
from tensorflow.keras.preprocessing.image import load_img

In [129]:
# Load models once with error handling
def load_models():
  # Load whisper-small model for Speech to Text conversion
    try:
        transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-small")
    except Exception as e:
        raise RuntimeError(f"Error loading ASR model: {e}")

  # Load spacy en_core_web_sm model for name extraction from a EN sentence
    try:
        nlp = spacy.load('en_core_web_sm')
    except Exception as e:
        raise RuntimeError(f"Error loading spaCy model: {e}")

  # Load custom CNN Model for Melanoma prediction
    try:
        cnn_model = tf.keras.models.load_model('gs://gh_gcs_bucket/melanoma_cnn_model_2.keras')
    except Exception as e:
        raise RuntimeError(f"Error loading CNN model: {e}")

  # Load Helsinki-NLP model for Spanish/French to English translator and vice versa
    try:
        translator_es_en = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")
        translator_fr_en = pipeline("translation_fr_to_en", model="Helsinki-NLP/opus-mt-fr-en")
        translator_en_es = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")
        translator_en_fr = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr")
    except Exception as e:
        raise RuntimeError(f"Error loading translation models: {e}")

    return transcriber, nlp, cnn_model, translator_es_en, translator_fr_en, translator_en_es, translator_en_fr

# Load models
transcriber, nlp, cnn_model, translator_es_en, translator_fr_en, translator_en_es, translator_en_fr = load_models()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [130]:
# Translate from Spanish or French to English
def translate_to_english(text, lang):
    try:
        if lang == "es":
            translated_text = translator_es_en(text)[0]['translation_text']
        elif lang == "fr":
            translated_text = translator_fr_en(text)[0]['translation_text']
        else:
            translated_text = text  # If the language is English or unsupported
        return translated_text
    except Exception as e:
        return f"Error translating text: {e}"

In [131]:
# Translate from English to Spanish or French
def translate_from_english(text, lang):
    try:
        if lang == "es":
            translated_text = translator_en_es(text)[0]['translation_text']
        elif lang == "fr":
            translated_text = translator_en_fr(text)[0]['translation_text']
        else:
            translated_text = text  # If the language is English or unsupported
        return translated_text
    except Exception as e:
        return f"Error translating text: {e}"

In [132]:
# Extract name from text
def extract_name(text):
    try:
        doc = nlp(text)
        name = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
        return name[0] if name else "No name found"
    except Exception as e:
        return f"Error extracting name: {e}"

In [133]:
# Get patient's image from Google cloud storage
def image_from_gcs(name):
    try:
        gcs_file_url = f'https://storage.googleapis.com/gh_patient_images/{name}.jpg'
        response = requests.get(gcs_file_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        return image, None
    except requests.exceptions.RequestException as e:
        return None, f"Error fetching image from GCS: {e}"
    except Exception as e:
        return None, f"Error processing image file: {e}"

In [134]:
# Normalize image
def prepare_image(image):
    try:
        # Resize image
        image = tf.image.resize(image, (300, 300))
        # image = image.resize((300, 300))
        # Normalize image
        image_normalized = np.array(image).astype(np.float32) / 255.0
        # Add batch dimension
        image_normalized = np.expand_dims(image_normalized, axis=0)
        return image_normalized, None
    except Exception as e:
        return None, f"Error preparing image: {e}"

In [135]:
# Model's Prediction
def predict(image_normalized):
    try:
        # Run through CNN Model for prediction
        prediction_probs = cnn_model.predict(image_normalized)
        # Get Max value from prediction probabilities
        prediction = np.argmax(prediction_probs, axis=1)
        confidence = np.max(prediction_probs)
        return prediction, confidence, None
    except Exception as e:
        return None, None, f"Error during prediction: {e}"

In [136]:
# Process Speech
def process_speech(audio):
    # Speech to text
    try:
        sr, y = audio
        y = y.astype(np.float32)
        y /= np.max(np.abs(y))
        text = transcriber({"sampling_rate": sr, "raw": y})["text"]
    except Exception as e:
        return f"Error during speech-to-text conversion: {e}", None

    # Language detection using langdetect library and translation to English
    try:
        detected_lang = detect(text)
        if detected_lang in ['es', 'fr']:
            text = translate_to_english(text, detected_lang)
    except Exception as e:
        return f"Error detecting or translating language: {e}", None

    # Extract name from text using spacy
    name = extract_name(text)
    if "Error" in name:
        return name, None
    if name == "No name found":
        return "No name found", None

    # Fetch image from Google Cloud Storage using the extracted name
    image, error = image_from_gcs(name)
    if error:
        return error, None

    # Prepare image for the model
    image_normalized, error = prepare_image(image)
    if error:
        return error, None

    # Get model prediction from CNN Model
    prediction, confidence, error = predict(image_normalized)
    if error:
        return error, None

    # Format the result
    if prediction == 1:
        result = f"Our model's prediction for {name} is malignant with {confidence*100:.2f}% confidence."
    else:
        result = f"Our model's prediction for {name} is benign with {confidence*100:.2f}% confidence."

    # Translate result to the original language if necessary
    if detected_lang in ['es', 'fr']:
        result = translate_from_english(result, detected_lang)

    # Return result and image only if successful
    return result, gr.update(value=image, visible=True)

In [137]:
# Add Image background in Gradio app, change color and size of Title and Description boxes

css_code = """
.gradio-container {
    background: url('https://www.healthcareitnews.com/sites/hitnai/files/GettyImages-949223490.jpg') no-repeat center center fixed;
    background-size: cover;
    height: 100vh;
    width: 100vw;
}

/* Styling the title (h1) */
.gradio-container h1 {
    color: #000000 !important;  /* Black color for the title */
    font-size: 63px;            /* Font size for the title */
    background-color: rgba(255, 255, 255, 0.8);  /* Semi-transparent background */
    padding: 20px;              /* Padding around the title */
    border-radius: 15px;        /* Rounded corners */
    border: 3px solid #4CAF50;  /* Green border */
    box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.5); /* Shadow effect */
}

/* Styling the description (p) */
.gradio-container p {
    color: #000000 !important;  /* Enforce black color for the description */
    font-size: 25px;            /* Font size for the description */
}

"""


In [138]:
# Doctor's AI assistant Gradio app
ai_assistant_app = gr.Interface(
    fn=process_speech,
    inputs=gr.Audio(sources="microphone"),
    outputs=[
        gr.Textbox(label="Predicted Results", lines=2),
        gr.Image(type="pil", label="Analyzed Image", visible=False)  # Hidden by default
    ],
    title="Physician's AI Assistant for Melanoma Detection",
    description="You can request results of one these patients - Barbara, Mario, Benjamin, Blake, Michael, Marco, Brooke, Maya, Barry, Mohan, Bob & Melodie. You can speak in English, Spanish or French.",
    css=css_code
)

In [139]:
# Launch the app
ai_assistant_app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://81532c526a264837d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
